In [25]:
import torch
import torch.nn.functional
import matplotlib
%matplotlib inline

In [26]:
# Get the vocabulary for the mappings of to and from intergers

words = open("names.txt",'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {v:k for k,v in stoi.items()}


In [35]:
# Build the dataset
X,Y = [],[]
context_length = 3

for w in words[:5]:
    context = [0] * context_length
    print(w)
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(f"{''.join(itos[i] for i in context)} ---> {itos[ix]}")
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [60]:
X.shape, X.dtype, Y.shape, Y.dtype


27

In [64]:
C = torch.randn((27,2))
C

tensor([[ 1.4140, -0.9164],
        [ 0.6345, -0.9186],
        [-0.4419,  0.8661],
        [-0.1640, -0.3767],
        [ 0.1695,  0.7452],
        [-2.2785, -0.4671],
        [ 0.1070,  0.6598],
        [-0.5363,  1.1308],
        [ 1.4783,  0.9955],
        [-0.0713,  0.4893],
        [ 0.6374,  0.9106],
        [ 1.3740, -2.3133],
        [ 1.2617, -1.1571],
        [ 1.6125,  0.0337],
        [ 0.1694, -1.0704],
        [ 1.4233,  0.2604],
        [-1.0649, -0.5052],
        [ 0.3267,  0.1601],
        [ 1.0696, -0.2487],
        [ 0.4330, -1.8828],
        [ 0.0865,  0.5502],
        [ 0.4356,  0.5052],
        [ 0.5720,  0.0388],
        [-0.8659, -0.2791],
        [-2.6366,  0.6593],
        [-1.1495, -0.4954],
        [ 0.0128, -0.3272]])

In [67]:
# Create the embeddings
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [82]:
# Create the hidden layer
W1 = torch.randn(6,100)
b1 = torch.randn(100)
h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1)

torch.Size([32, 100])

In [84]:
# Create the output layer
W2 = torch.randn(100,27)
b2 = torch.randn(27)

logits = h @ W2 + b2
